This Jupyter Notebook accompanies the paper "Mukai lifting of self-dual points" by Barbara Betti and Leonie Kayser.
The code is in Julia language.
The folder $\texttt{MukaiLiftSource}$ contains all the files and we need to work inside this directory.
We describe how to use the package $\texttt{MukaiLiftP6}$.

In [10]:
using Pkg;
Pkg.activate("MukaiLiftP6")

  Activating project at `~/Documents/GitHub/Gr26/MukaiLiftSource/MukaiLiftP6`


In [11]:
using MukaiLiftP6

We define the configuration of points as a $7\times 14$ matrix $\Gamma$. 
We prove that it is self-dual by computing the matrix $\Lambda$ satisfying condition $(ii)$ of Lemma $2.1$ and checking that $\Gamma\cdot\Lambda\cdot\Gamma^\intercal=0$.

In [12]:
Gamma = [7 -2 6 -1 -6 1 -9 7 0 6 1 8 -3 7; -1 2 -5 -2 0 -4 3 -3 -4 -3 4 -2 4 -1; 1 4 -1 -5 -3 6 8 -1 -8 -3 5 1 -6 -8; 3 -6 4 -3 -4 6 0 5 8 2 3 2 -8 0; 1 -2 1 0 -4 2 2 3 4 -1 2 2 -2 -2; 0 -6 -5 6 3 7 -3 2 8 -7 -6 -3 -5 5; -3 3 -4 1 4 3 2 -3 -6 -4 -3 -4 -1 -2]
Lambda = certify_selfdual(Gamma)
norm( Gamma*Lambda*transpose(Gamma), Inf) #1.7763568394002505e-14

UndefVarError: UndefVarError: `certify_selfdual` not defined

In [9]:
Pkg.status()

Project MukaiLiftP6 v0.1.0
Status `~/Documents/GitHub/Gr26/MukaiLiftSource/MukaiLiftP6/Project.toml`
  [8bb1440f] DelimitedFiles v1.9.1
  [f213a82b] HomotopyContinuation v2.9.3
  [4138dd39] JLD v0.13.5
  [f1435218] Oscar v1.0.3
  [14b8a8f1] PkgTemplates v0.7.50
  [37e2e46d] LinearAlgebra


We compute the Orthogonal normal form $\Gamma_{\text{ONF}}= [ I | P]$ of $\Gamma$ together with the linear projective transformation $A$ such that $A\cdot \Gamma_{\text{ONF}}= \Gamma\cdot \Lambda_{\text{scale}}$.

In [4]:
Gamma_ONF, A, Lambda_scale = normalize_SApoints(Gamma)
norm(A*Gamma_ONF - Gamma*Lambda_scale, Inf)

2.220446049250313e-15

The orthogonal normal form of $\Gamma$ is given by $[I | P]$, where $P$ is the orthogonal matrix:

In [5]:
P = Matrix{Float64}(Gamma_ONF[:,8:14])

7×7 Matrix{Float64}:
 -0.339885   0.059965   -0.0924461  …   0.756392   -0.303323    0.391105
 -0.205214  -0.850587    0.0196552      0.26801     0.19422    -0.292163
  0.100775   0.230199    0.811776       0.314816   -0.13109    -0.320813
  0.094778   0.318981   -0.408428       0.329141    0.303156   -0.0306488
 -0.773382   0.0614641   0.155243      -0.374698   -0.160906    0.200883
  0.26268   -0.166903   -0.242692   …  -0.071958   -0.855751   -0.155243
 -0.395103   0.294162   -0.286833       0.0549679  -0.0546348  -0.770301

We compute the Telen norma form $\Gamma_{\text{TNF}}$ of $\Gamma$ using the Cayley transform of $P$.

In [8]:
S = cayley_num(P)
Gamma_TNF = [I+S I-S]

7×14 Matrix{Float64}:
  1.0           -1.0   1.0  -1.0623e-14  …    1.0623e-14   1.0   -4.0   -2.0
  1.0            1.0   3.0  -2.0              2.0          2.0  -10.0  -12.0
 -1.0           -3.0   1.0   2.0             -2.0         -1.0    1.0    2.0
 -6.26471e-15    2.0  -2.0   1.0              1.0          1.0   10.0    6.0
  1.0            2.0  -1.0   1.0             -1.0          1.0    4.0    4.0
 -4.0          -10.0   1.0  10.0         …  -10.0         -4.0    1.0    6.0
 -2.0          -12.0   2.0   6.0             -6.0         -4.0   -6.0    1.0

The target parameters $S_{\text{target}}$ of the homotopy are the entries of the skew-symmetric matrix $S$.

In [9]:
S_target = skew_to_vector(S)

21-element Vector{Any}:
 -0.9999999999999816
  0.9999999999999984
 -1.0623014660713928e-14
 -1.000000000000007
  3.999999999999994
  2.000000000000003
  2.999999999999989
 -2.000000000000049
 -2.00000000000003
  9.99999999999996
  ⋮
  1.0000000000000049
 -0.9999999999999925
 -2.0000000000000013
 -0.9999999999999869
 -9.999999999999995
 -6.000000000000012
 -3.9999999999999893
 -3.9999999999999964
 -5.999999999999995

We solve the mukai lifting problem for $\Gamma$.

In [10]:
@time result = HomotopyContinuation.solve(parametrized_system, l_start; compile = false, start_parameters=S_start, target_parameters=S_target)

9154.619175 seconds (99.77 M allocations: 39.423 GiB, 0.05% gc time, 0.44% compilation time)


Result with 1 solution
• 1 path tracked
• 1 non-singular solution (0 real)
• random_seed: 0x2da3bfc3


We analyze the solution. There is one path tracked with $1429$ steps. 
The residual is the infinity norm of the vector with the equations evaluated in the computed solution and it is numerically zero.

In [12]:
sol = solutions(result)[1]
result.path_results

1-element Vector{PathResult}:
 PathResult:
 • return_code → :success
 • solution → ComplexF64[-0.25682992026101414 - 0.35352382958954476im, 0.07969833194871812 - 0.11571018739124789im, -0.09803109836006561 - 0.10914260579734429im, 0.10845446432945789 + 0.1213834741397078im, 0.07054820840239133 + 0.12372182781912008im, 0.034322032342818026 + 0.1287167678979342im, 0.10725708940751046 + 0.4343412410485293im, -0.29792718455039213 - 0.13280424269497115im, -0.07658176136159506 - 0.27576406865601im, -0.09985492528086015 + 0.02190214225580802im  …  -0.15447266899745862 - 0.09892023294732782im, -0.008805556396581355 + 0.1911048519825844im, 0.0030346242894781082 - 0.17269615245001688im, -0.23137949583152712 - 0.2546612734594592im, -0.010953640238052193 - 0.0934920698599431im, -0.02222878869905518 + 0.19759109538418715im, -0.1405859227250611 - 0.0526613568904753im, 0.16367653788044578 - 0.29963375910458334im, -0.3184718168060914 - 0.1617058275439371im, -0.19788431876656168 - 0.01693724052181073im

The embedding $\widehat L: \mathbb{P}^6\simeq \mathbb{L}\subseteq \mathbb{P}^{14} $ such that $\widehat L(\Gamma_{\text{TNF}})=\mathbb{L}\cap X_8$ is 

In [13]:
L_hat = L_start + sum(sol[i]*A_rand[i] for i in eachindex(l))

15×7 Matrix{ComplexF64}:
   -2.39867-0.760585im      0.136927-0.0731426im   …  -0.0549966+1.54306im
   0.120381-0.687571im      0.118839+0.0175647im        0.609849-0.349064im
  -0.874564+0.489957im     -0.133184+0.161753im        -0.484225-0.170822im
  0.0900887-0.80893im       0.213471+0.539043im        -0.336838+0.803372im
  -0.660979-1.1459im       0.0312071+0.382946im        -0.554239+0.55146im
    1.91901+0.50087im      0.0779703+0.333896im    …     -1.3462+0.21279im
  -0.266978-0.832334im     -0.255807-0.494672im         -2.21711+0.997617im
   -0.22492-3.90594im      -0.635566-0.176003im         -1.23474-1.53634im
  -0.408724-0.897665im     -0.341599-0.401544im         -1.70067-0.732123im
    1.33645+1.10583im    -0.00976897+0.302164im         0.557198+0.715766im
  -0.113212-0.0633213im     0.419823-0.176421im    …    0.508407+0.838852im
 -0.0102615+0.157629im      0.208108+0.00825777im       0.733529+0.933496im
  -0.187805-0.131456im      0.350144+0.326324im          1.62697+0.

Composing with the previous coordinate transformations $\Gamma \leadsto \Gamma_{\text{TNF}}$ we obtain $L \colon \mathbb{P}^6 \to \mathbb{P}^{14}$ such that $L(\Gamma)= \mathbb{L}\cap X_8$

In [14]:
L = L_hat*(I+S)*inv(A)

15×7 Matrix{ComplexF64}:
  -2.43381-4.45071im    -2.86825-4.49986im   …    1.85726-4.12053im
 -0.571879+1.44484im   -0.213834+1.77774im       -2.17004+0.819291im
  0.495441-0.523548im    1.01963-0.898375im       2.63775+0.589267im
  0.958401-2.80583im    0.350694-3.20325im      -0.734016-3.48259im
  0.505669-2.34915im   0.0221898-2.41816im        1.21409-1.60625im
   2.02096-3.16752im     1.04499-4.93784im   …    2.92458-2.06121im
   5.18851+1.67306im     4.03309+4.42388im         5.8118-0.786245im
   3.18389+1.1179im      4.84222+1.73233im        8.57393+9.09686im
   3.93173+1.05756im     4.70829+0.929709im       8.69331+3.16074im
  -1.17617-1.68854im    -1.02489-2.27274im       -1.17072-3.32841im
  -2.54915-0.570031im   -3.52059+0.245008im  …   -2.94125-1.78315im
  -2.25506-0.596842im   -2.57807+0.322309im       -2.8166-2.26911im
  -1.30788-2.48251im    -0.16074-2.09195im       -6.09222-1.63651im
  -1.31124-1.72178im   -0.557748-1.70476im       -3.54458-0.648492im
  0.239302+0.778868

We can verify $L(\Gamma) = \mathbb{L} \cap X_8$ by evaluating $ L(\gamma_i)$ in the Plücker relations.

In [15]:
q = poly_to_fp(plück_oscar)
maximum([ norm(q(L*Gamma[:,i]) ,Inf) for i =1:14])

1.2805801671064476e-11